In [ ]:
from dash import Dash, dcc, html, Input, Output, dash_table
import pandas as pd
import dash_bootstrap_components as dbc
import dash
import numpy as np


#df = pd.read_excel(r"C:\Users\delil\OneDrive\Desktop\Mydatabase.xlsx")

df = pd.read_csv(r"Mydatabase_CSV.csv")
# must be a CSV or searches with numbers will be output a result

# simplex was used because it had the nice gray background
# this will not work without callback exceptions. I think it is because all pages are ran at the same time
# they are ran at the same time because of the == path because they are all connected


instruction = [html.Br(),html.Br(),html.Br(),'The Aging Metabolite Database is a manually curated database that includes data',
                ' from 100 published papers that focus on metabolites and how the concentration of these metabolites', 
                ' change with age.  It can be used to compare new studies with previously published studies.', 
               html.Br(),html.Br(),
                'The database includes information about the metabolite, such as the metabolite name,', 
                ' InChIKey, Keggs, Lipid Maps, Pubchem CID, Standardized Name, Formula, Exact Mass, Subclass, and HMDB.',
                html.Br(), html.Br(),
                'It also includes information about the paper, such as the organ, sex, and species tested,', 
                ' if the metabolite increased or decreased with age, and the DOI.',
                html.Br(), html.Br(),
                'Single searches display every instance that the metabolite has been reference while batch searches', 
                ' display how many times a metabolite has been referenced.']




app = Dash(external_stylesheets=[dbc.themes.SIMPLEX],suppress_callback_exceptions=True)

# dcc.storage default is storage_type = 'memory' this means the data is store until the page is refreshed

app.layout = html.Div([dcc.Location(id='page_location'),
                       dcc.Store(id='singleStored'),
                       dcc.Store(id='batchstore'), 
                       dcc.Store(id='ss2'),
                       dcc.Store(id='bs2'),
# nav bar makes the overall title at the top
                       dbc.NavbarSimple([ 
# navlink is the link to each of the tabs on the navbar
                    dbc.NavLink('Single Search', href='/',style = {'color': 'white'}), 
                    dbc.NavLink('Batch Search', href='/batch',style = {'color': 'white'}),
                    dbc.NavLink('Metabolite Frequency', href='/frequency',style = {'color': 'white'}),
                    dbc.NavLink('Download csv', href='/download',style = {'color': 'white'})],      
                    brand='Aging Metabolite Database - AMDB', color = '#1A3E68', links_left = 'True', 
                    brand_style = {'color': '#FFCD00'},style={'width':'170%','margin-left': '-44%'}),
                       
                    dbc.Container(id='container_of_page')], style = {'margin-left': '-22%','width':'150%'})


# this is where the paths are selected. If single search is clicked it ouputs what the divs include

@app.callback(Output('container_of_page', 'children'), [Input('page_location', 'pathname')])
def pages(pathname):
    
# / is the path name they much has a backslash
    
    if pathname == '/':
        
        return html.Div(children = [html.H3('Search for information about a single metabolite',
                    style = {'textAlign':'center','margin-left': '-17%','padding': '1.1%'}), 
                    html.Div([dcc.Dropdown(id='singledropdown',
                            options=[{'label': i, 'value': i} for i in df],value='HMDB')],
                             style={'width': '17%','margin-left': '-7%','margin-top': '1.5%'}),
 # the rows dont control what is typed but rather what you can see
                    dcc.Textarea(id = 'singleTextarea', rows = '1',
                    placeholder='Enter a metabolite here', spellCheck='true',
                    style={'width': '60%','textAlign':'left', 'color': 
                       '#696969','margin-left': '12%', 'padding': '1.1%','margin-top': '-3%'}), html.Br(), html.Br(),
                         html.Button(dcc.Link('Submit', href='/page-2', 
                               style = {'color':'black','text-decoration': 'none'}),style = {'margin-left': '12%'}),
                                    html.H3(instruction,style={'width': '120%','font-size': '20px','margin-left': '-17%'})
])
                            

    if pathname == '/page-2':
        
        return html.Div(html.Div(id='textareaout', 
                                 style={'whiteSpace': 'pre-line','margin-left': '-20%','padding': '1.1%'}))

    
    if pathname == '/batch':
        
        return html.Div([html.H3('Batch search for multiple metabolites',
                    style = {'textAlign':'center','margin-left': '-15%','padding': '1.1%'}), 
                 html.Div([dcc.Dropdown(id='batchdropdown',
                            options=[{'label': i, 'value': i} for i in df],value='HMDB')],
                         style={'width': '17%','margin-left': '-7%','outline':'blue'}),        
                dcc.Textarea(id = 'batchsearch',
                    placeholder='Enter multiple metabolites here', spellCheck='true',
                    style={'width': '60%', 'height': '300px','textAlign':'left', 'color': 
                       '#696969','padding': '1.1%','margin-left': '12%','margin-top': '-2.5%'}),
                html.Br(), html.Br(),
                    html.Button(dcc.Link('Submit', href='/batch2',
# text decoration none is because there is an underline by default for links
                    style = {'color':'black','text-decoration': 'none'}),style = {'margin-left': '12%'})
])
       

    if pathname == '/batch2':
        
        return html.Div(html.Div(id='batchout', style={'whiteSpace': 'pre-line','margin-left': '-20%','padding': '1.1%'}))
                         
    if pathname == '/frequency':
#padding puts space right after the div so it isnt so close        
        return html.Div([html.H3(
            'The following chart shows how often each metabolite appears',style={'padding': '1.1%','margin-left': '12%'}),
                         html.Div([dcc.Dropdown(id='demo-dropdown',
                            options=[{'label': i, 'value': i} for i in df],value='HMDB',
                            style={'width': '100%','background-color': '#F2F2F2'})], 
                            style={'width': '17%','margin-left': '-7%'}),
                       html.Div(id='dd-output-container',
                            style={'margin-left': '12%','margin-top': '-3%','width':'70%','font-size': '18px'})
])
    
    if pathname == "/download":
        
        return html.Div([html.H3('Download the database',
                        style={'padding': '1.1%','margin-left': '30%'}),
# I have only seen records/name/id used so I think its manditory
                         html.Div([dash_table.DataTable(df.to_dict('records'),[{'name': i, 'id': i} for i in df],
                            export_format="csv",
                            style_cell={'textAlign': 'left','font-family':'Open Sans','background-color': '#F2F2F2'},
                            style_data={'maxWidth':'275px'})],
                        style={'margin-left': '-18%','margin-top': '-2.5%'})
])
                           
@app.callback(
    Output('dd-output-container', 'children'),
    Input('demo-dropdown', 'value')
)
def update_output(value):
    frequency = df[value].value_counts()
# frequency takes in the value (selected column) and does the group by
    data = {'Count': frequency}
# data is the output that will be put into the dataframe
    indexReset = pd.DataFrame(data).reset_index()
# the value column will dissapear as it is the index column so it has to be reset 
# reset is when the new index is 0,1,2 instead of citrate, protein, etc
    finalOutput = indexReset.rename(columns={'index': value})
# dash only allows dictionaries to be the datatable so it has to be converted 
    return dash_table.DataTable(finalOutput.to_dict('records'),[{'name': i, 'id': i} for i in finalOutput],
                            export_format="csv",
                            style_cell={'textAlign': 'left','font-family':'Open Sans','background-color': '#F2F2F2'})
            

@app.callback(
        Output('singleStored','data'),
        [Input('singleTextarea','value')])

@app.callback(
        Output('batchstore','data'),
        [Input('batchsearch','value')])

@app.callback(
        Output('ss2','data'),
        [Input('singledropdown', 'value')])

@app.callback(
        Output('bs2','data'),
        [Input('batchdropdown', 'value')])
        
# this function just takes the typed words from the textArea and stores it into memory
def return_name(value):
    return value


@app.callback(
    Output('textareaout', 'children'),
    [Input('singleStored', 'data'),Input('ss2','data')])



def singleSearch(data,value):
    lists = list(data.split("\n"))
    a = np.array([])
    b = np.array([])
    c = np.array([])
    d = np.array([])
    e = np.array([])
    f = np.array([])
    g = np.array([])
    h = np.array([])
    p = np.array([])
    j = np.array([])
    k = np.array([])
    l = np.array([])
    m = np.array([])
    n = np.array([])
    o = np.array([])
    for i in lists:
        a = np.append(a,df[df[value].str.contains(i,case=False)==True]['Metabolite Name'].to_numpy())
        b = np.append(b,df[df[value].str.contains(i,case=False)==True]['increased or decreased'].to_numpy())
        c = np.append(c,df[df[value].str.contains(i,case=False)==True]['Organ'].to_numpy())
        d = np.append(d,df[df[value].str.contains(i,case=False)==True]['Sex'].to_numpy())
        e = np.append(e,df[df[value].str.contains(i,case=False)==True]['Age'].to_numpy())
        f = np.append(f,df[df[value].str.contains(i,case=False)==True]['Species'].to_numpy())     
        g = np.append(g,df[df[value].str.contains(i,case=False)==True]['InChIKey'].to_numpy())
        h = np.append(h,df[df[value].str.contains(i,case=False)==True]['Keggs'].to_numpy())
        p = np.append(p,df[df[value].str.contains(i,case=False)==True]['Lipid Maps'].to_numpy())
        j = np.append(j,df[df[value].str.contains(i,case=False)==True]['Standardized Name'].to_numpy())
        k = np.append(k,df[df[value].str.contains(i,case=False)==True]['Formula'].to_numpy())
        l = np.append(l,df[df[value].str.contains(i,case=False)==True]['Exact mass'].to_numpy())
        m = np.append(m,df[df[value].str.contains(i,case=False)==True]['Subclass'].to_numpy())
        n = np.append(n,df[df[value].str.contains(i,case=False)==True]['HMDB'].to_numpy())
        o = np.append(o,df[df[value].str.contains(i,case=False)==True]['DOI'].to_numpy())
    columns = {'Metabolite Name': a,
            'Increased or Decreased': b,
            'Organ': c,
            'Sex': d,
            'Age': e,
            'Species': f,
            'InChIKey': g,
            'Keggs': h,
            'Lipid Maps': p,
            'Standardized Name': j,
            'Formula': k,
            'HMDB': l,
            'Exact Mass': m,
            'HMDB': n,
            'DOI': o}
    dataframe = pd.DataFrame(columns) 
             
    return dash_table.DataTable(dataframe.to_dict('records'),[{'name': i, 'id': i} for i in dataframe],export_format="csv",
                               style_cell={'textAlign': 'left','font-family':'Open Sans','background-color': '#F2F2F2'})
                     



@app.callback(
    Output('batchout', 'children'),
    [Input('batchstore', 'data'),Input('bs2','data')])

# this function takes the text area that was stored on the new page, turns it into a list
# and splits it by row so each row is in its own string
# ['a','b','c']
# this is possible beacause textarea puts an extra space at the end of a row so the split '__'
# this also allows for a space in the name

def batchSearch(data,value):
    lists = list(data.split("\n"))
    a = np.array([])
    b = np.array([])
    c = np.array([])
    d = np.array([])
    e = np.array([])
    f = np.array([])
    g = np.array([])
    h = np.array([])
    p = np.array([])
    j = np.array([])
    k = np.array([])
    l = np.array([])
    m = np.array([])
    n = np.array([])
    o = np.array([])
    for i in lists:
        a = np.append(a,df[df[value].str.contains(i,case=False)==True]['Metabolite Name'].to_numpy())
        b = np.append(b,df[df[value].str.contains(i,case=False)==True]['increased or decreased'].to_numpy())
        c = np.append(c,df[df[value].str.contains(i,case=False)==True]['Organ'].to_numpy())
        d = np.append(d,df[df[value].str.contains(i,case=False)==True]['Sex'].to_numpy())
        e = np.append(e,df[df[value].str.contains(i,case=False)==True]['Age'].to_numpy())
        f = np.append(f,df[df[value].str.contains(i,case=False)==True]['Species'].to_numpy())     
        g = np.append(g,df[df[value].str.contains(i,case=False)==True]['InChIKey'].to_numpy())
        h = np.append(h,df[df[value].str.contains(i,case=False)==True]['Keggs'].to_numpy())
        p = np.append(p,df[df[value].str.contains(i,case=False)==True]['Lipid Maps'].to_numpy())
        j = np.append(j,df[df[value].str.contains(i,case=False)==True]['Standardized Name'].to_numpy())
        k = np.append(k,df[df[value].str.contains(i,case=False)==True]['Formula'].to_numpy())
        l = np.append(l,df[df[value].str.contains(i,case=False)==True]['Exact mass'].to_numpy())
        m = np.append(m,df[df[value].str.contains(i,case=False)==True]['Subclass'].to_numpy())
        n = np.append(n,df[df[value].str.contains(i,case=False)==True]['HMDB'].to_numpy())
        o = np.append(o,df[df[value].str.contains(i,case=False)==True]['DOI'].to_numpy())
    columns = {'Metabolite Name': a,
            'Increased or Decreased': b,
            'Organ': c,
            'Sex': d,
            'Age': e,
            'Species': f,
            'InChIKey': g,
            'Keggs': h,
            'Lipid Maps': p,
            'Standardized Name': j,
            'Formula': k,
            'HMDB': l,
            'Exact Mass': m,
            'HMDB': n,
            'DOI': o}
    dataframe = pd.DataFrame(columns) 
    frequency = dataframe[value].value_counts()
#    frequency = dataframe.value_counts()
    newdata = {'Count': frequency}
    indexReset = pd.DataFrame(newdata).reset_index()
             
    return dash_table.DataTable(indexReset.to_dict('records'),[{'name': i, 'id': i} for i in indexReset],
                            export_format="csv",
                            style_cell={'textAlign': 'left','font-family':'Open Sans','background-color': '#F2F2F2'}) 



# use_reloader is needed when ran on jupyter or debug wont work

                                              
app.run_server(debug=True, use_reloader=False, host="0.0.0.0") 

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: on
